# PhotoGuard API
GPU-accelerated image protection using PhotoGuard.
Exposes an API endpoint via ngrok for Oracle server integration.

In [ ]:
# Install dependencies
!pip install flask pyngrok pillow torch torchvision diffusers transformers

In [ ]:
# Clone PhotoGuard repo
!git clone https://github.com/photoguard/photoguard.git
%cd photoguard

In [ ]:
from flask import Flask, request, jsonify, send_file
from pyngrok import ngrok
import os
import uuid

app = Flask(__name__)

@app.route('/health', methods=['GET'])
def health():
    return jsonify({'status': 'healthy', 'service': 'photoguard-colab'})

@app.route('/api/photoguard/protect', methods=['POST'])
def protect_image():
    if 'image' not in request.files:
        return jsonify({'error': 'No image provided'}), 400
    
    file = request.files['image']
    job_id = str(uuid.uuid4())
    
    # Save input
    input_path = f'/tmp/{job_id}_input.png'
    output_path = f'/tmp/{job_id}_output.png'
    file.save(input_path)
    
    # TODO: Run PhotoGuard processing
    # For now, placeholder
    import shutil
    shutil.copy(input_path, output_path)
    
    return jsonify({
        'job_id': job_id,
        'status': 'completed',
        'download_url': f'/api/photoguard/download/{job_id}'
    })

@app.route('/api/photoguard/download/<job_id>', methods=['GET'])
def download(job_id):
    path = f'/tmp/{job_id}_output.png'
    if os.path.exists(path):
        return send_file(path)
    return jsonify({'error': 'Not found'}), 404

In [ ]:
# Start ngrok tunnel
ngrok.set_auth_token('YOUR_NGROK_TOKEN')
public_url = ngrok.connect(5000)
print(f'PhotoGuard API available at: {public_url}')

# Run Flask
app.run(port=5000)